In [1]:
import pandas as pd

pd.read_csv('')

,annotation_id,annotator,content,created_at,id,lead_time,link_berita,nama_berita,sentiment,tanggal_berita,title,updated_at
0,3,1,Kumaha barudak? Well!Yuk ramaikan Pesta Anak N...,2024-03-22T02:29:28.095843Z,1,45.971,https://20.detik.com/advertorial/20231109-2311...,Detik News,Neutral,10-11-2023,PAN x JakCloth! Pesta Anak Nongkrong Sambangi ...,2024-03-22T02:29:37.860861Z
